In [28]:
from bs4 import BeautifulSoup as bs
import requests
import random
import time
import re
import pandas as pd

In [77]:
def get_price(product):
    product_price = 'N/A'
    try:
        product_price = product.find('p', {'class', 'price'}).text
    except AttributeError:
        print('missing price')
    #print('product_price', product_price)
    if product_price.find('$') != -1:
        p = product_price.split("$")
        car_price = p[1]
        price = int(car_price.replace(',' , ''))
        return price
    else:
        return product_price

In [78]:
def get_title(product):
    product_title = 'N/A'
    try:
        product_title = product.find('p', {'class', 'title'}).text
    except AttributeError:
        print('missing title')
    #print('product_title', product_title)
    return product_title

In [103]:
def get_seller_info(product):
    #print("product", product)
    #print('product type', type(product))
    product_info = 'N/A'
    try:
        product_info = product.find('p', {'class', 'dealer'}).text
    except AttributeError:
        print('missing info')
    info = product_info.split(",")
    return info

In [104]:
def get_features(product):
    product_features = 'N/A'
    try:
        product_features = product.find('ul', {'class', 'features'}).text
    except AttributeError:
        print('missing feature')
    #print('product_feature type', type(product_features))
    #print('product_feature', product_features)
    return product_features

In [105]:
def split_feature(features):
    #print("features", features)
    #print('features type', type(features))
    feature = features.split("\n")
    #print("feature", feature)
    #print('feature type', type(feature))
    #print('feature length', len(feature))
    listBodytype = ["4X4", "Convertible", "Coupe", "Hatchback", "People Mover", "Sedan", "Small Truck", "SUV", "Ute", "Van And Minivan", "Wagon"]
    listTransmission = ["Automatic", "Continuous Variable", "Manual", "Tiptronic"]
    listcolour = ["Beige", "Black", "Blue", "Bronze", "Brown", "Gold", "Green", "Grey", "Maroon","Orange","Pink", "Purple", "Red", "Silver", "White", "Yellow"]
    listfueltype = ["Diesel", "Dual Fuel", "Electric", "Hybrid", "Lpg", "Petrol", "Unleaded"]
    
    mileage = 'N/A'
    doors = 'N/A'
    engine = 'N/A'
    carfuel = 'N/A'
    cartype = 'N/A'
    cartransmission = 'N/A'
    carcolour = 'N/A'
    for i in range (len(feature)):
        f = feature[i]
        #print("i", i)
        #print("f", f)
        #print('type of f', type(f))
        if f.find('km') != -1:
            p = f.split("km")
            mileage
            mileage_comma= p[1]
            mileage = f
        if f.find('Door') != -1:
            #print("f", f)
            cardoors = f.split("Door")
            #print("cardoors", cardoors)
            cardoor = cardoors[0].split(" ")
            #print("cardoor", cardoor)
            doors = cardoor[-2]
        if f.find('cc') != -1:
            sizes = f.split("cc")
            size = sizes[0].split(" ")
            engine = size[-1]
        for bodytype in listBodytype:
            if f.find(bodytype) != -1:
                cartype = bodytype
        for transmission in listTransmission:
            if f.find(transmission) != -1:
                cartransmission = transmission
        for colour in listcolour:
            if f.find(colour) != -1:
                carcolour = colour
        for fueltype in listfueltype:
            if f.find(fueltype) != -1:
                carfuel = fueltype
    #print ("mileage", mileage)
    #print ("doors", doors )
    #print ("engine", engine)
    #print ("carfuel", carfuel)
    #print ("cartype", cartype)
    #print ("cartransmission", cartransmission)
    #print ("carcolour", carcolour)
    features_dict = {
        "mileage": mileage,
        "doors": doors,
        "engine": engine,
        "carfuel": carfuel,
        "cartype": cartype,
        "cartransmission": cartransmission,
        "carcolour": carcolour,
    }
    #for x in features_dict:
      #print(x)
    return features_dict

In [116]:
def scrap(url, sort_by, header):
    #sort_by = &sort=MostPopular
    product_dict = {}
    
    index = 1
    for page in range(1,10):
        wait_time = random.uniform(7,16)
        time.sleep(wait_time)
        print ("Scraping: " + url + '?page=' + str(page) + sort_by)
        response = requests.get(url +  '?page=' + str(page) + sort_by, headers=header)
        html = response.content
        page_soup = bs(html, 'lxml')
        products = page_soup.find_all('div', {'class' : 'list-item '})
            
        if products is None:
            print('No products found')
            break
            
        
        rank_count = 1
        for product in products:
            #gets year brand and model
            title = get_title(product)
            t = title.split(" ", 2)
            features = get_features(product)
            a = split_feature(features)            
            mileage = a["mileage"]
            miles = mileage.split("km", 1)[0] 
            miles = miles.replace(',', '')
            miles = int(miles)
            b = get_seller_info(product)
            sellertype = b[0]
            sellerlocation = b[1].lstrip().title()
            #for x in a:
              #print(x)
            #print("type a",type(a))
            #print("a",a)
            #f = features.split("\n")
            product_dict[index] = {
                    "year" : t[0],
                    "brand" : t[1],
                    "model" : t[2],
                    "price" : get_price(product),
                    "mileage" : miles,
                    "door_number": a["doors"],
                    "engine_size": a["engine"],
                    "fuel_type": a["carfuel"],
                    "bodystyle": a["cartype"],
                    "transmission": a["cartransmission"],
                    "colour": a["carcolour"],
                    "seller_type": sellertype,
                    "seller_location": sellerlocation,
                    #"product_mileage" : f[1],
                    #"product_feature" : f[2],
                    #"product_engine" : f[3],
                    #"product_fuel" : f[4],
                    #"brand_name" : brand,
                    # "current_price" : price_info["current price"],
                    # "original_price" : price_info["original price"],
                    #  "discounted" : price_info["discounted"],
                    #  "number_of_reviews" : get_product_review_number(product),
                    #  "number_of_stars" : get_product_number_of_stars(product),
                    #  "product_shipping" : get_shipping_information(product),
                    "page" : page,
                    "page_rank" : rank_count
                    }
            rank_count = rank_count + 1
            index = index + 1
            #print ("---------------------------------------------------------------------------------")
        print("Number of products scrapped: " + str(len(product_dict)))
        break # limit one page      
    return product_dict         

In [117]:
base_url = "http://www.autotrader.co.nz/used-cars-for-sale"
sort_by = "&sort=MostPopular"

# do get request using request library
header={'user-agent':''}

response = requests.get(base_url, headers=header) # requires user aganet header

#check if response was successful - expect 200 back if successful
#print(response.status_code)

#check content
#print(response.content)

#stop if not 200
if response.status_code != 200:
    print("Error: " + response.status_code)
    exit(0)
else:
    print("Response: " + str(response.status_code))
    
    #print html to see if getting anything back
print(response)


products = scrap(base_url, sort_by, header)

print("Total number of products scraped: " + str(len(products)))
for keys,values in products.items():
    print(keys)
    print(values)

Response: 200
<Response [200]>
Scraping: http://www.autotrader.co.nz/used-cars-for-sale?page=1&sort=MostPopular
Number of products scrapped: 20
Total number of products scraped: 20
1
{'year': '1998', 'brand': 'BMW', 'model': '528I', 'price': 9995, 'mileage': 45101, 'door_number': '4', 'engine_size': '2800', 'fuel_type': 'Petrol', 'bodystyle': 'Sedan', 'transmission': 'Automatic', 'colour': 'N/A', 'seller_type': 'Dealer', 'seller_location': 'Wellington', 'page': 1, 'page_rank': 1}
2
{'year': '2004', 'brand': 'BMW', 'model': '525I', 'price': 11999, 'mileage': 117065, 'door_number': '4', 'engine_size': '2500', 'fuel_type': 'Petrol', 'bodystyle': 'Sedan', 'transmission': 'Tiptronic', 'colour': 'N/A', 'seller_type': 'Dealer', 'seller_location': 'Hawkes Bay', 'page': 1, 'page_rank': 2}
3
{'year': '2004', 'brand': 'BMW', 'model': '530I', 'price': 17999, 'mileage': 161684, 'door_number': '4', 'engine_size': '3000', 'fuel_type': 'Petrol', 'bodystyle': 'Sedan', 'transmission': 'Automatic', 'colo

In [118]:
# write results to a file
df = pd.DataFrame.from_dict(products, orient='index')
df.to_csv('./output-test-20.csv')